In [1]:
import cv2
import numpy as np
from collections import Counter

# Recursive function to create Shannon-Fano codes
def shannon_fano_coding(symbols, code=""):
    if len(symbols) == 1:
        symbol, _ = symbols[0]
        return {symbol: code}
    
    # Find the split point
    total = sum([freq for _, freq in symbols])
    cumulative = 0
    split_index = 0
    
    for i, (_, freq) in enumerate(symbols):
        cumulative += freq
        if cumulative >= total / 2:
            split_index = i
            break
    
    # Split symbols into two parts and assign '0' or '1' to each part
    left_symbols = symbols[:split_index+1]
    right_symbols = symbols[split_index+1:]
    
    codes = {}
    codes.update(shannon_fano_coding(left_symbols, code + "0"))
    codes.update(shannon_fano_coding(right_symbols, code + "1"))
    
    return codes

# Encode the image using Shannon-Fano coding
def shannon_fano_encoding(image, codes):
    encoded_image = "".join(codes[pixel] for row in image for pixel in row)
    return encoded_image

# Decode the Shannon-Fano encoded string back to an image
def shannon_fano_decoding(encoded_image, codes, shape):
    reverse_codes = {v: k for k, v in codes.items()}
    current_code = ""
    decoded_image = []
    
    for bit in encoded_image:
        current_code += bit
        if current_code in reverse_codes:
            decoded_image.append(reverse_codes[current_code])
            current_code = ""
    
    return np.array(decoded_image).reshape(shape)

# Main function to apply Shannon-Fano coding to an image
def main():
    # Load the image in grayscale mode
    image = cv2.imread('lenna.jpeg', cv2.IMREAD_GRAYSCALE)
    
    # Step 1: Calculate Frequency
    frequency = Counter(image.flatten())
    sorted_freq = sorted(frequency.items(), key=lambda item: item[1], reverse=True)
    
    # Step 2: Generate Shannon-Fano Codes
    codes = shannon_fano_coding(sorted_freq)
    
    # Step 3: Encode Image
    encoded_image = shannon_fano_encoding(image, codes)
    
    # Step 4: Decode Image
    decoded_image = shannon_fano_decoding(encoded_image, codes, image.shape)
    
    # Display the original and decoded images
    cv2.imshow('Original Image', image)
    cv2.imshow('Decoded Image', decoded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
